In [1]:
from OSMPythonTools.overpass import Overpass
overpass = Overpass()

Посмотреть на карту можно здесь: https://www.openstreetmap.org/#map=1/70/0

Документация: https://wiki.openstreetmap.org/wiki/Overpass_API/Language_Guide#Background_and_concepts

Существует три типа объектов:
 * node — узел (город, населенный пункт, страна)
 * way — путь
 * rel — relation, отношения между nodes

Помимо них, есть area, который задается немного иначе синтаксически, но имеет название (например, страна, город) и выдает список всех имеющихся внутри или как-либо относящихся к области объектов. Объекты можно фильтровать как и при обычном запросе. Эти запросы не всегда содержат самую обновленную информацию, но через них доступ к большому кол-ву объектов, якобы, осуществляется быстрее. Area отличается от node тем, что последний, как формат данных, не включает в себя никакое множество объектов, только описывает этот, конкретный.

Запрос состоит минимум из двух частей:
* отвечает за тип объектов, фильтры
* отвечает за вывод и количество/тип информации о найденных объектах

Пример представлен ниже. Части запроса разделяются точкой с запятой. Можно задавать несколько запросов первого типа и один для вывода, вернется объединение.
После выполнения запроса нужно преобразовать результат в элементы.

In [147]:
result = overpass.query('node["name"="Берлин"];out body;')
results = result.elements()

In [148]:
results[0].tag('name:en')

'Berlyn'

## Tag request clauses (or "tag filters")

  * ["key"]            /* filter objects tagged with this key and any value */
  * [!"key"]           /* filter objects not tagged with this key and any value */
  * ["key"="value"]    /* filter objects tagged with this key and this value */
  * ["key"!="value"]   /* filter objects tagged with this key but not this value, or not tagged with this key */
  * ["key"~"value"]    /* filter objects tagged with this key and a value matching a regular expression */
  * ["key"!~"value"]   /* filter objects tagged with this key but a value not matching a regular expression */
  * [\~"key"~"value"]   /* filter objects tagged with a key and a value matching regular expressions */
  * [\~"key"~"value",i] /* filter objects tagged with a key and a case-insensitive value matching regular expressions */

## Bounding box clauses ("bbox query", "bounding box filter")

Bounding box clauses (like all other clauses for filters or recursion queries) can only be used as filters after specifying a main query type (or .resultset), they are not queries by themselves. In the Overpass QL syntax, they have a form like in this example:

  /*your query here*/(51.0, 7.0, 52.0, 8.0)
  
Bounding box clauses always start with the lowest latitude (southernmost) followed by lowest longitude (westernmost), then highest latitude (northernmost) then highest longitude (easternmost). Note that this is different from the ordering in the XAPI syntax.

## Area clauses ("area filters")

* are not directly OSM objects but a bunch of them located in one area
* they are generated (and cached on the Overpass API server) by a batch running periodically on this server to process all new or modified data changes in the database so the data in them is not always the newest
* their objects include closed ways, or relations (notably "boundary" and "multipolygon" relations) whose members include one or more ways joined together to create closed "inner" or "outer" rings delimiting a surface
* they can greatly simplify and speedup the queries, notably for large surfaces with complex geometries, usually using "boundary" or "multipolygon" relations (such as boundaries of countries or their regional subdivisions, or the complex boundaries of large "landuse" or "natural" areas)

These areas can then be used as more selective bounding filters (instead of using simple bounding boxes), or can also be used themselves as queries (in which case they will return all nodes in the surface enclosed by the area geometry).

Чтобы сделать запрос по множеству внутри area необходимо:

* написать запрос, чтобы задать area:  area[name="Russia"];)->.a;
* прописать условие как к обычному запросу рядом с (area.a)

Почему-то у меня оно не работает так, как я ожидала: он выдает какие-то объекты с названием Russia, но не все объекты, связанные со страной. Надо еще разобраться.

In [29]:
result = overpass.query('(area[name="Russia"]["place"="country"];)->.a; out body;')
results = result.elements()

In [30]:
results[0].__dict__

IndexError: list index out of range

In [10]:
result = overpass.query('(area[name="Россия"];)->.a;nwr["name"="Москва"](area.a); out body;')
results = result.elements()

[overpass] downloading data: [timeout:25][out:json];(area[name="Россия"];)->.a;nwr["name"="Москва"](area.a); out body;
Exception: [overpass] runtime error: Query timed out in "query" at line 1 after 26 seconds.
NoneType: None
[overpass] error in result (cache/overpass-8080894d20387033c3cdfa7ae4dc4c189663d515): [timeout:25][out:json];(area[name="Россия"];)->.a;nwr["name"="Москва"](area.a); out body;
NoneType: None


Exception: [overpass] error in result (cache/overpass-8080894d20387033c3cdfa7ae4dc4c189663d515): [timeout:25][out:json];(area[name="Россия"];)->.a;nwr["name"="Москва"](area.a); out body;

In [ ]:
len(results)

Then this named result set can be used to filter objects by intersection with this area. The following filter clause will keep only objects from the current result set that are intersecting with any area present in the named result set "a":

/*your query here*/(area.a)

Note that this filter will keep any object (nodes, ways, relations, or areas) in the main query that are fully contained in any area in "a", or partially covered by any one of them, or any object in the query whose surface completely encloses any area in "a". For example, if the selected area is the area of a city, the filter will return all local subdivisions (districts, quarters) in that city, all natural features (such as lakes) in that city, and all subdivisions that are including that city (nodes, ways, relations and areas in the input set), unless you use additional tag filters in the main query to be more selective (such as ["admin_level"="8"] for just selecting OSM objects representing that city; the closed objects for the surrounding cities that also tagged with ["admin_level"="8"], and that are sharing only a common border with the selected city will normally not be returned because the surface of their mutual intersection should be empty and limited to these common borders; however any nodes and ways that are in the input set, and that are also falling exactly on the border of the selected area will be returned, independently of other relations or closed ways in which they could be members).

## Recursion clauses ("recursion filters")

Or recursion forward or backward about membership links

  * (r)
  * (w)
  * (n)
  * (br)
  * (bw)
  * (bn)
  * (>)
  * (>>)
  * (<)
  * (<<)

## Special clauses ("special filters")

Or special clauses of the form "(type:value)", e.g. the "(around:value)" clause.

## Features & tags

Информация о тегах при объектах: https://wiki.openstreetmap.org/wiki/Map_features

## Вывод

* geom: Add the full geometry to each object. This adds coordinates to each node, to each node member of a way or relation, and it adds a sequence of "nd" members with coordinates to all relations.
* bb: Adds only the bounding box of each element to the element. For nodes this is equivalent to "geom". For ways it is the enclosing bounding box of all nodes. For relations it is the enclosing bounding box of all node and way members, relations as members have no effect.
* center: This adds only the center of the above mentioned bounding box to ways and relations. Note: The center point is not guaranteed to lie inside the polygon (example).

## Координаты

Возможные проблемы:
* может понадобиться специфицировать какие-то детали, например, тип населенного пункта или bounding box, в случае, если по названию выдастся больше одного узла

In [6]:
result = overpass.query('node["name"="Тетея"]; out geom;')
results = result.elements()

In [7]:
results[0].__dict__

{'_json': {'type': 'node',
  'id': 2001367584,
  'lat': 60.98027,
  'lon': 106.702499,
  'tags': {'name': 'Тетея',
   'place': 'hamlet',
   'population': '37',
   'population:date': '2010-10-14',
   'wikidata': 'Q4456586',
   'wikipedia': 'ru:Тетея'}},
 '_soup': None,
 '_shallow': False}

## Пути

Каждый узел имеет уникальный индекс. Путь представляет собой последовательный список индексов узлов. Пока что не понятно, как задать условие на выбор пути таким образом, чтобы были соединены два узла.

In [15]:
result = overpass.query('node["name"="Москва"]["place"="city"]; out geom;')
results = result.elements()
len(results)

1

In [9]:
results[0].__dict__['_json']['id']

1686293227

In [10]:
result = overpass.query('node["name"="Тула"]["place"="city"]; out geom;')
results = result.elements()
len(results)

1

In [11]:
results[0].__dict__['_json']['id'] # просто индексы узлов

34389350

In [45]:
result = overpass.query('way["name"="Москва"]; out geom;')
results = result.elements()
len(results)

192

In [47]:
results[0].__dict__['_json']['tags'] # какой-то рандомный путь

{'boat': 'no',
 'name': 'Москва',
 'name:en': 'Moscow',
 'name:ru': 'Москва',
 'waterway': 'river',
 'wikidata': 'Q175117',
 'wikipedia': 'ru:Москва (река)'}

In [38]:
results[0].__dict__['_json']['nodes'] # узлы в нем

[259964812,
 640536924,
 824421587,
 824422134,
 824421252,
 824421680,
 824422230,
 259964813,
 7302530178,
 824421337,
 7302530177,
 259964814,
 721036835,
 1395138813,
 7302530153,
 259964815,
 1113597785,
 7302530119,
 1395138816,
 7302530154,
 259964816,
 7302530156,
 7302530155,
 7302530157,
 259964817,
 7302530159,
 7302530158,
 7302530160,
 824421909,
 7302530162,
 7302530161,
 7302530163,
 259964818,
 7302530164,
 824422328,
 1113597719,
 7302530175,
 824421478,
 7302530176,
 259964819,
 259964820,
 259964821,
 7303935938,
 7303935939,
 259964822,
 7303935940,
 395390667,
 7303935941,
 259964823,
 7303935942,
 824422401,
 7303935943,
 395390671,
 7303935944,
 259964824,
 7303935945,
 824421970,
 824421571,
 7303935946,
 7303935947,
 259964825,
 1080453525,
 7303935948,
 824422232,
 7303935949,
 824421684,
 824422204,
 7303987757,
 7303987756,
 7303987758,
 259964826,
 7303987755,
 824421654,
 824421140,
 7303987759,
 7303935969,
 7303989970,
 7303935970,
 259964827,
 730393597

Можно посчитать длину пути функцией length, но есть проблемы. Первая заключается в том, что нам нужен только сегмент пути, и пока непонятно, как его вырезать. О второй ниже.

Вот описание функции и ее применение из документации:

Length
since v0.7.55

The length operator returns the length of the element. For ways this is the length of the way. For relations this is the sum of the lengthes of the members of type way. For nodes it is always zero.

Its syntax is:

 length()

В средней части запроса мы посчитали сумму длин путей, вывели их, создав новый объект stat.

In [69]:
result = overpass.query('way["name"="Москва"]; make stat length=sum(length()); out;')
results = result.elements()
len(results)

[overpass] downloading data: [timeout:25][out:json];way["name"="Москва"]; make stat length=sum(length()); out;


1

In [70]:
results[0].__dict__

{'_json': {'type': 'stat', 'id': 1, 'tags': {'length': '563877.511'}},
 '_soup': None,
 '_shallow': False}

Хочется попробовать вывести просто длину каждого пути. Для этого задаем цикл по сету, пока что просто с выводом.

In [93]:
result = overpass.query("""way["name"="Москва"] -> .a;
                            foreach.a -> .b(
                            .b out;
                            );""")
results = result.elements()
len(results)

192

Как только пытаемся прицепить функцию, все ломается. Почему-то внутри цикла элемент не передается в функцию length самостоятельно, а если передать ей элемент вручную (вписать length(.b)), то все ломается, и Overpass API ругается, что этой функции нельзя передавать аргументы. Пробовала тестить здесь: https://overpass-turbo.eu/


In [92]:
result = overpass.query("""way["name"="Москва"] -> .a;
                            foreach.a -> .b(
                            make stat length=length();
                            out;
                            );""")
results = result.elements()
len(results)

[overpass] downloading data: [timeout:25][out:json];way["name"="Москва"] -> .a;
                            foreach.a -> .b(
                            length() out;
                            );
The requested data could not be downloaded. HTTP Error 400: Bad Request
Traceback (most recent call last):
  File "C:\Users\VADIK\anaconda3\lib\site-packages\OSMPythonTools\internal\cacheObject.py", line 83, in __query
    response = urllib.request.urlopen(request)
  File "C:\Users\VADIK\anaconda3\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\VADIK\anaconda3\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "C:\Users\VADIK\anaconda3\lib\urllib\request.py", line 640, in http_response
    response = self.parent.error(
  File "C:\Users\VADIK\anaconda3\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "C:\Users\VADIK\anaconda3\lib\urllib\request.py", line 502, in _call_

Exception: The requested data could not be downloaded. HTTP Error 400: Bad Request